In [9]:
!df -h

In [2]:
import os
os.environ['HF_HOME'] = '/d/cache'

In [ ]:
import minsearch

In [ ]:
import json 

In [ ]:
with open('documents.json') as f_in:
    docs_raw = json.load(f_in)

In [ ]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [ ]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [ ]:
index.fit(documents)

In [7]:
from openai import OpenAI

In [ ]:
client = OpenAI()

In [ ]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [9]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [5]:
tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-xl')
model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-xl', device_map='auto')

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Trying to resume download...
Error while downloading from https://cdn-lfs.huggingface.co/repos/16/41/16418edd56a7c42307a0f361531c01ee227a92a98628972bd433062c276dad7c/99196ddfbe886e8ef860f52de979df64890edfc792c3d94ce0502991f347dd18?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model-00001-of-00002.safetensors%3B+filename%3D%22model-00001-of-00002.safetensors%22%3B&Expires=1720219135&Policy=eyJTdGF0ZW1l

KeyboardInterrupt: 

In [ ]:
input_text = "translate English to German: What is your name?"

In [8]:
input_ids = tokenizer(input_text, return_tensors='pt').input_ids.to('cuda')
outputs = model.generate(input_ids)
result = tokenizer.decode(outputs[0])

result

In [ ]:
def build_prompt(query, search_results):
    prompt_template = """
        You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
        Use only the facts from the CONTEXT when answering the QUESTION.

        QUESTION: {question}

        CONTEXT: {context}
    """.strip()

    context_str = ""

    for doc in search_results:
        context_str = context_str + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context_str).strip()
    return prompt 

def llm(prompt):
    input_ids = tokenizer(prompt, return_tensors='pt').input_ids.to('cuda')
    outputs = model.generate(input_ids)
    result = tokenizer.decode(outputs[0])

    return result

In [ ]:
def llm(prompt, generate_params=None):
    if generate_params is None:
        generate_params = {}

    input_ids = tokenizer(prompt, return_tensors='pt').input_ids.to('cuda')
    outputs = model.generate(
        input_ids, 
        max_length=generate_params.get("max_length", 100),
        num_beams=generate_params.get("num_beams", 5),
        do_sample=generate_params.get("do_sample", False),
        temperature=generate_params.get("temperature", 1.0),
        top_k=generate_params.get("top_k", 50),
        top_p=generate_params.get("top_p", 0.95),
    )
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result